In [160]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib import_ipynb

In [161]:
from fastai.tabular.all import *
from pathlib import Path
import import_ipynb
import ipynb
import stockFetcher

In [162]:
testFolder = Path.cwd().parent / 'TestData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

In [163]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
countryGuesser.pkl
stockScreenerV2.1.pkl
stockScreenerV2.0.pkl
stockScreenerV1.5.pkl
animalClassifier.pkl
stockScreenerV1.4.pkl
stockScreenerV1.6.pkl
leagueChampionClassifier.pkl
stockScreenerV1.7.pkl
politicianClassifier.pkl
stockScreenerV1.3.pkl
stockScreenerV3.0.pkl
stockScreenerV1.2.pkl
stockScreenerV1.0.pkl
stockScreenerV1.1.pkl
test.pkl
stockScreenerV1.9.pkl
stockScreenerV1.8.pkl
stockScreenerV1.10.pkl


In [164]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[0]
bestModel.head()

Model Name    stockScreenerV2.1
Timestamp      2025-01-19 14:33
MAE                       0.205
RMSE                      0.284
R2                       -0.331
Name: 12, dtype: object

## Load model

In [165]:
modelName = f"{bestModel['Model Name']}.pkl" # Change this if you want to try other models
learn = load_learner(modelFolder / modelName)

## Predictions

In [174]:
predictionTarget = 'BBW' # 'Best' for the best performing tickers 

if predictionTarget == 'Best':
    adr_df = pd.read_csv(testFolder / 'tickers.csv')
    symbols = adr_df['Ticker'].tolist()
    test_df = [stockFetcher.getTickerData(symbol) for symbol in symbols]
    test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame

else:
    test_df = stockFetcher.getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
    test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermed

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,38.57,39.400002,38.259998,39.099998,201700,0.0,0.0,8.308939,5.255001e+08,0.095078,Specialty Retail


In [176]:
if predictionTarget == 'Best':
    prediction = learn.get_preds(dl=dl)
    sorted_predictions = sorted(zip(symbols, prediction[0]), key=lambda x: x[1], reverse=True)
    print(f"Prediction for best performing tickers:")
    for symbol, pred in sorted_predictions:
        print(f"{symbol}: {pred[0].item() * 100:.2f}%")
else:
    prediction = learn.get_preds(dl=dl)
    print(f"Prediction for {predictionTarget}:")
    print(f"{prediction[0][0][0].item() * 100:.2f}%")

print("Free money?!")

Prediction for BBW:
3.24%
Free money?!
